# Import Needed Libraries

In [34]:
from statistics import mean,stdev
import ndlib.models.ModelConfig as mc
from termcolor import colored
import ndlib.models.epidemics as ep
import time
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [35]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)    
    return first_nodes

In [36]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)   
    return bottom_nodes

In [37]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

# SI Simulations for all Networks

In [38]:
#networks = next(walk(RealNetworks), (None, None, []))[2]

for network in networks:
    g = nx.read_gml(f'{RealNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]
    
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))
    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraphs(max)
    #print(network)
    
    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])
        
    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/RealNetworks/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here
    
    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_averageTime_each_simulation'] = dict()
        globals()[f'{centrality}_standardTime_each_simulation'] = dict()
        globals()[f'{centrality}_averageIterations_each_simulation'] = dict()
        globals()[f'{centrality}_standardIterations_each_simulation'] = dict()
        globals()[f'{centrality}_averageIterationsHalfPath_each_simulation'] = dict()
        globals()[f'{centrality}_standardIterationsHalfPath_each_simulation'] = dict()
        for fraction in fractions: 
            globals()[f'{centrality}_averageTime_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standardTime_each_simulation'][fraction] = 0
            globals()[f'{centrality}_averageIterations_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standardIterations_each_simulation'][fraction] = 0
            globals()[f'{centrality}_averageIterationsHalfPath_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standardIterationsHalfPath_each_simulation'][fraction] = 0
    
    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')
    
    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')

    
    for centrality in colors.keys(): 
        print(colored(f'Centrality: {centrality}', 'green',attrs=['bold']))
        for fraction in fractions:
            globals()[f'time_{centrality}'] = list()
            globals()[f'iterations_{centrality}'] = list()
            globals()[f'iterationsHalfPath_{centrality}'] = list()
            fraction_infected = round(fraction*network_size)
            if(centrality=="modv_neg"):
                infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
            else:
                infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
                
            
            
            for i in range(repeatedSimulations): 
                start = time.time()
                model = ep.SIModel(g)
                config = mc.Configuration()
                config.add_model_parameter('beta', epidemicThreshold+0.1)      
                config.add_model_initial_configuration("Infected", infected_nodes)
                model.set_initial_status(config)

                x= 0
                halfPath = 0
                flag = 0
                iterations = model.iteration_bunch(1)

                while((iterations[x]['node_count'][1])!=network_size):
                    if(len(infected_nodes)==0):
                        x = 0 
                        halfPath = 0
                        break
                    #print(flag)
                    if((iterations[x]['node_count'][1]>=(network_size/2)) and flag==0):
                        flag = 1
                        halfPath = x
                        #print(f"Half Path: {halfPath} {i}")
                    x = x + 1
                    iterations = iterations + model.iteration_bunch(1)
                    
                end = time.time()
                nbIterations = x
                takenTime = end - start
                globals()[f'time_{centrality}'].append(takenTime)
                globals()[f'iterations_{centrality}'].append(nbIterations)
                globals()[f'iterationsHalfPath_{centrality}'].append(halfPath)
                
            globals()[f'{centrality}_averageIterationsHalfPath_each_simulation'][fraction] = mean(globals()[f'iterationsHalfPath_{centrality}'])
            globals()[f'{centrality}_standardIterationsHalfPath_each_simulation'][fraction] = stdev(globals()[f'iterationsHalfPath_{centrality}'])
            globals()[f'{centrality}_averageTime_each_simulation'][fraction] = mean(globals()[f'time_{centrality}'])
            globals()[f'{centrality}_standardTime_each_simulation'][fraction] = stdev(globals()[f'time_{centrality}'])
            globals()[f'{centrality}_averageIterations_each_simulation'][fraction] = mean(globals()[f'iterations_{centrality}'])
            globals()[f'{centrality}_standardIterations_each_simulation'][fraction] = stdev(globals()[f'iterations_{centrality}'])
            print(fraction,":",mean(globals()[f'time_{centrality}']))
            print(fraction,":",mean(globals()[f'iterations_{centrality}']))
        e = epidemicThreshold + 0.1
        print(epidemicThreshold)
        p = f"./Pickle Variables/{n}/beta-{e}/"
        if(path.exists(p)==False):
            os.mkdir(p)
        save_obj(globals()[f'{centrality}_averageTime_each_simulation'], p,f'{centrality}_averageTime_each_simulation')
        save_obj(globals()[f'{centrality}_standardTime_each_simulation'], p, f'{centrality}_standardTime_each_simulation') 
        save_obj(globals()[f'{centrality}_averageIterations_each_simulation'], p,f'{centrality}_averageIterations_each_simulation')
        save_obj(globals()[f'{centrality}_standardIterations_each_simulation'], p, f'{centrality}_standardIterations_each_simulation') 
        save_obj(globals()[f'{centrality}_averageIterationsHalfPath_each_simulation'], p,f'{centrality}_averageIterationsHalfPath_each_simulation')
        save_obj(globals()[f'{centrality}_standardIterationsHalfPath_each_simulation'], p, f'{centrality}_standardIterationsHalfPath_each_simulation') 

Simulation currently on network 911allwords.gml : Name: 
Type: Graph
Number of nodes: 13308
Number of edges: 148035
Average degree:  22.2475...
Centrality: modv_pos
0.01 : 3.6982772755622864
0.01 : 73.61
0.02 : 3.675899705886841
0.02 : 73.74
0.03 : 3.5850674700737
0.03 : 72.1
0.04 : 3.585732057094574
0.04 : 72.39
0.05 : 3.6537171959877015
0.05 : 74.39
0.06 : 3.5032839250564574
0.06 : 70.64
0.07 : 3.5965255522727966
0.07 : 73.1
0.08 : 3.5200418424606323
0.08 : 71.38
0.09 : 3.5590410351753237
0.09 : 72.16
0.1 : 3.6303184628486633
0.1 : 74.13
0.11 : 3.588176965713501
0.11 : 73.37
0.12 : 3.55036806344986
0.12 : 72.69
0.13 : 3.4682886886596678
0.13 : 70.68
0.14 : 3.531266305446625
0.14 : 71.63
0.15 : 3.438861267566681
0.15 : 69.98
0.16 : 3.468762047290802
0.16 : 71.39
0.17 : 3.512237935066223
0.17 : 71.63
0.18 : 3.5180847787857057
0.18 : 72.32
0.19 : 3.450908763408661
0.19 : 71.15
0.2 : 3.4193582677841188
0.2 : 71.71
0.21 : 3.4752261447906494
0.21 : 71.98
0.22 : 3.4623615646362307
0.22 : 72

0.07 : 3.4459022092819214
0.07 : 73.79
0.08 : 3.291460008621216
0.08 : 72.24
0.09 : 3.43581387758255
0.09 : 73.05
0.1 : 3.3603780484199524
0.1 : 71.45
0.11 : 3.370714240074158
0.11 : 72
0.12 : 3.3263159728050233
0.12 : 71.55
0.13 : 3.4296814727783205
0.13 : 73.91
0.14 : 3.2501731061935426
0.14 : 70.91
0.15 : 3.3420063161849978
0.15 : 71.8
0.16 : 3.35994891166687
0.16 : 72.25
0.17 : 3.3297854137420653
0.17 : 71.52
0.18 : 3.3188504123687745
0.18 : 70.09
0.19 : 3.3815642690658567
0.19 : 72.64
0.2 : 3.309867913722992
0.2 : 70.63
0.21 : 3.380541763305664
0.21 : 72.27
0.22 : 3.309215371608734
0.22 : 70.56
0.23 : 3.3873371982574465
0.23 : 72.78
0.24 : 3.324238350391388
0.24 : 71.33
0.25 : 3.3957393980026245
0.25 : 72.4
0.26 : 3.311462321281433
0.26 : 70.73
0.27 : 3.18913090467453
0.27 : 68.12
0.28 : 3.173610928058624
0.28 : 68.81
0.29 : 3.182230088710785
0.29 : 68.96
0.3 : 3.0927797865867617
0.3 : 67.16
0.31 : 3.0896933889389038
0.31 : 67.13
0.32 : 3.102053303718567
0.32 : 66.8
0.33 : 3.17929

0.18 : 3.126184437274933
0.18 : 70.71
0.19 : 3.0721855115890504
0.19 : 72.01
0.2 : 3.032200937271118
0.2 : 70.86
0.21 : 3.1634860944747927
0.21 : 73.59
0.22 : 3.032270803451538
0.22 : 71.37
0.23 : 3.0282000637054445
0.23 : 71.39
0.24 : 2.986854693889618
0.24 : 70.47
0.25 : 3.0467021679878235
0.25 : 71.44
0.26 : 3.0640122413635256
0.26 : 70.95
0.27 : 3.0709561419487
0.27 : 70.74
0.28 : 3.1161661338806153
0.28 : 72.13
0.29 : 3.0236975026130675
0.29 : 70
0.3 : 3.0498065400123595
0.3 : 70.96
0.31 : 2.982082006931305
0.31 : 68.95
0.32 : 3.1385379004478455
0.32 : 73.14
0.33 : 3.02787606716156
0.33 : 70.66
0.34 : 2.9931848740577696
0.34 : 69.51
0.35 : 3.080677320957184
0.35 : 71.35
0.36 : 3.065461058616638
0.36 : 70.63
0.37 : 3.1306276059150697
0.37 : 72.82
0.38 : 3.1244597864151
0.38 : 72.86
0.39 : 3.118306133747101
0.39 : 71.85
0.4 : 3.1364510226249696
0.4 : 71.41
0.41 : 3.130987374782562
0.41 : 71.64
0.42 : 3.0906184363365172
0.42 : 70.44
0.43 : 3.015959641933441
0.43 : 70.33
0.44 : 3.0725